### Analytics Project - Baseball Pitch by Pitch Analysis
#### Sikyun (George) Lee 
1. Obtain a Dataset
2. Model and store dataset (SQL, Pandas, ORM, Python Objects, etc.)
3. Visualize (Relationship, Insights, Descriptive Statstistics)
4. Think ahead of project
5. Due 11/25


#### Background information on data
Data is downloaded from Kaggle (https://www.kaggle.com/pschale/mlb-pitch-data-20152018?select=atbats.csv) but the source of this is from a webscrape in (http://gd2.mlb.com/components/game/mlb/). 

This data looks at pitch-level data from the 2015 to 2018 seasons in MLB. Through this data, I am hoping to answer some of these questions from a pitching standpoint.

**1. What are the pitch-types that gets outs the most? (unhittable pitches)**\
**2. Which pitcher has most pitches per inning ratio? (shows efficiency)**\
**3. Some other questions will asked as project progresses**\

For this session, I will some simple Exploratory Data Analysis to better understand and visualize the data

#### 1. Data source will be from the above ***mlb.com*** source through webscraping if possible. Currently, the link above is dead and if another substitute link cannot be found, the dataset from Kaggle will be used.

In [2]:
#For homework purposes, I'll only import and work with the 2019 season data

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#quick test
test = pd.read_csv('20tor.txt')

In [ ]:
test.head(10)

#### 2. Store Data as Pandas Objects

In [ ]:
atbats = pd.read_csv('2019_atbats.csv')
games = pd.read_csv('2019_games.csv')
pitches = pd.read_csv('2019_pitches.csv')
names = pd.read_csv('player_names.csv')

#### 3. Visualize the Data for:
- Exploratory Data Analysis to find relationships and insights
- Descriptive Statistics

In [ ]:
atbats.columns

In [ ]:
games.columns

In [ ]:
pitches.columns

In [ ]:
# What pitches were thrown the most frequently
# Join the pitches pitch_type column by the ab_id to the atbats data
joined_atbats = atbats.copy(deep = True)

In [ ]:
joined_atbats = pd.merge(joined_atbats, pitches, how = 'left', on = 'ab_id')

In [ ]:
joined_atbats.columns

In [ ]:
a_viz = joined_atbats[['inning', 'top', 'ab_id', 'g_id', 'p_score', 'batter_id', 'pitcher_id',
             'p_throws', 'o', 'zone', 'pitch_type', 'b_count', 's_count', 'outs', 
              'pitch_num', 'on_1b', 'on_2b', 'on_3b']]

In [ ]:
a_viz1 = a_viz['o_lag'] = a_viz['o'].shift(1)

In [ ]:
a_viz['out'] = a_viz['o'] - a_viz['o_lag']

In [ ]:
#Let's see who has on average and median thrown the most innings
a_viz1 = a_viz.groupby(by = 'pitcher_id', as_index=False).agg([np.mean])

In [ ]:
a_viz1['pitcher_id'] = a_viz1.index

In [ ]:
names['full_name'] = names.first_name + " " + names.last_name

In [ ]:
#names = names.rename(columns = {'pitcher_id' : 'id'})

In [ ]:
a_viz1['avg_inning'] = a_viz1['inning', 'mean']

In [ ]:
a_viz1.sort_values('avg_inning', ascending=False)

In [ ]:
a_viz1 = a_viz1.rename(columns = {'pitcher_id' : 'id'})

In [ ]:
a_viz2 = a_viz1.head(10)

In [ ]:
a_viz3 = pd.merge(a_viz2, names, how = 'left', on = 'id')

In [ ]:
a_viz3['avg_inning'] = a_viz3['inning', 'mean']

In [ ]:
a_viz3 = a_viz3.sort_values('avg_inning', ascending=False)

#### Output 1: Top 10 Pitchers with Highest Avg. Innings

In [ ]:
# Barplot of top 10 pitchers with highest avg innings
sns.set(font_scale=2) 
f, ax = plt.subplots(figsize=(30, 15))
ax = sns.barplot(x="full_name", y="avg_inning", data=a_viz3)
ax.set(ylabel = "Avg. Innings", xlabel = "Pitcher", title = 'Top 10 Pitchers with Highest Avg. Innings')

In [ ]:
b_viz1 = a_viz.groupby(by = ['pitch_type'], as_index=False).count()

In [ ]:
b_viz1['counts'] = b_viz1['inning']

In [ ]:
b_viz1 = b_viz1.sort_values('counts', ascending = False)

#### Output 2: Countplot of Pitch Types Thrown in 2019 Season

In [ ]:
# Barplot of frequently thrown pitches in MLB 2019 season
sns.set(font_scale=3) 
f, ax = plt.subplots(figsize=(30, 15))
ax = sns.barplot(x="pitch_type", y="counts", data=b_viz1)
ax.set(ylabel = "Total Pitch Count", xlabel = "Pitch Type", title = 'Frequently Thrown Pitches')

In [ ]:
pitch_out = a_viz.groupby(by = ['pitch_type'], as_index = False).sum()

In [ ]:
pitch_out_viz = pitch_out[['pitch_type', 'out']]

In [ ]:
pitch_out_viz = pitch_out_viz.sort_values('out')

#### Output 3: Pitch Types that Converted an Out

In [ ]:
# Barplot of frequently thrown pitches in MLB 2019 season
sns.set(font_scale=3) 
f, ax = plt.subplots(figsize=(30, 30))
ax = sns.barplot(x="pitch_type", y="out", data=pitch_out_viz)
ax.set(ylabel = "Total Outs Made", xlabel = "Pitch Type", title = 'Total Outs Converted by Pitch Types')

#### 4. Analysis 

#### 4-1. Comparing Speed Changes by Pitch Type to Strikes
For each pitch type:
- How much does speed change? (This may be obvious for some types)
- How does the change of speed affect the number of strikes? (compared to balls, outs)

In [ ]:
speed = joined_atbats[['inning','ab_id','g_id','batter_id','pitcher_id','event','start_speed','end_speed','spin_rate','spin_dir',
               'break_angle','break_length','break_y','ax','ay','az','zone','pitch_type','b_score','b_count',
               's_count','outs','pitch_num','on_1b','on_2b','on_3b']]

In [ ]:
speed['out'] = speed['outs'] - speed['outs'].shift(1)
#a_viz['out'] = a_viz['o'] - a_viz['o_lag']
#a_viz1 = a_viz['o_lag'] = a_viz['o'].shift(1)

In [ ]:
speed['change_speed'] = speed['start_speed'] - speed['end_speed']

In [ ]:
#change of speed per pitcher per game
speed_change_pitcher_game = speed.groupby(by = ['g_id','pitcher_id','pitch_type','event'])['change_speed'].mean()

In [ ]:
speed_viz1 = speed_viz.reset_index()

#df.pivot_table(index='number', columns='class').swaplevel(axis=1).sort_index(1)#

In [ ]:
speed_viz2 = speed.groupby(by = ['event'],as_index=False)['change_speed'].agg([np.mean]).reset_index()

In [ ]:
event_count = speed.groupby(by = ['event'],as_index=False).count().reset_index()

In [ ]:
event_count = event_count[['event','outs']]

In [ ]:
event_count['counts'] = event_count['outs']

In [ ]:
speed_viz2['event_cleaned'] = 0

speed_viz2.loc[0, 'event_cleaned'] = "Out"
speed_viz2.loc[1, 'event_cleaned'] = "Out"
speed_viz2.loc[2, 'event_cleaned'] = "Out"
speed_viz2.loc[3, 'event_cleaned'] = "Out"
speed_viz2.loc[4, 'event_cleaned'] = "Not Out"
speed_viz2.loc[5, 'event_cleaned'] = "Out"
speed_viz2.loc[6, 'event_cleaned'] = "Out"
speed_viz2.loc[7, 'event_cleaned'] = "Out"
speed_viz2.loc[8, 'event_cleaned'] = "Not Out"
speed_viz2.loc[9, 'event_cleaned'] = "Out"
speed_viz2.loc[10, 'event_cleaned'] = "Not Out"
speed_viz2.loc[11, 'event_cleaned'] = "Not Out"
speed_viz2.loc[12, 'event_cleaned'] = "Out"
speed_viz2.loc[13, 'event_cleaned'] = "Out"
speed_viz2.loc[14, 'event_cleaned'] = "Out"
speed_viz2.loc[15, 'event_cleaned'] = "Out"
speed_viz2.loc[16, 'event_cleaned'] = "Not Out"
speed_viz2.loc[17, 'event_cleaned'] = "Out"
speed_viz2.loc[18, 'event_cleaned'] = "Out"
speed_viz2.loc[19, 'event_cleaned'] = "Not Out"
speed_viz2.loc[20, 'event_cleaned'] = "Not Out"
speed_viz2.loc[21, 'event_cleaned'] = "Not Out"
speed_viz2.loc[22, 'event_cleaned'] = "Out"
speed_viz2.loc[23, 'event_cleaned'] = "Not Out"
speed_viz2.loc[24, 'event_cleaned'] = "Out"
speed_viz2.loc[25, 'event_cleaned'] = "Out"
speed_viz2.loc[26, 'event_cleaned'] = "Out"
speed_viz2.loc[27, 'event_cleaned'] = "Out"
speed_viz2.loc[28, 'event_cleaned'] = "Out"
speed_viz2.loc[29, 'event_cleaned'] = "Out"
speed_viz2.loc[30, 'event_cleaned'] = "Not Out"
speed_viz2.loc[31, 'event_cleaned'] = "Out"
speed_viz2.loc[32, 'event_cleaned'] = "Out"
speed_viz2.loc[33, 'event_cleaned'] = "Out"
speed_viz2.loc[34, 'event_cleaned'] = "Out"
speed_viz2.loc[35, 'event_cleaned'] = "Out"
speed_viz2.loc[36, 'event_cleaned'] = "Out"
speed_viz2.loc[37, 'event_cleaned'] = "Out"
speed_viz2.loc[38, 'event_cleaned'] = "Not Out"
speed_viz2.loc[39, 'event_cleaned'] = "Not Out"
speed_viz2.loc[40, 'event_cleaned'] = "Out"
speed_viz2.loc[41, 'event_cleaned'] = "Out"
speed_viz2.loc[42, 'event_cleaned'] = "Not Out"
speed_viz2.loc[43, 'event_cleaned'] = "Out"
speed_viz2.loc[44, 'event_cleaned'] = "Not Out"
speed_viz2.loc[45, 'event_cleaned'] = "Not Out"

In [ ]:
speed_viz3 = pd.merge(speed_viz2, event_count, how="left", on ='event')

In [ ]:
speed_viz3.drop(columns = ['outs'], inplace=True)

In [ ]:
speed_viz3.sort_values('mean',ascending=False).head()

In [ ]:
# Barplot of frequently thrown pitches in MLB 2019 season

sns.set(font_scale=3) 
f, ax = plt.subplots(figsize=(30, 15))
ax = sns.barplot(x="event", y="mean", data=speed_viz3, hue = 'event_cleaned')
ax.set(ylabel = "Total Count of Events", title = 'Number of Events')
ax.set_xticklabels(labels = speed_viz3['event'], rotation=45, fontsize=14)
ax.set(xlabel = "Events")

#Seems as though there isn't any big difference between speed change in Outs and Not Out results

### Part 2: Building Functions to Automatically Calculate Player Stats
1. Average Pitches thrown per game + visualization per pitcher per game
2. Average Strikes and Balls + Ratio by pitcher per game + visualization
3. Average Ground balls, Fly balls, Hits, On-base, Outs/On-Base Ratio by pitcher per game + visualization
4. Average Start_speed and End_speed, Start_speed:End_speed Ratio by Pitcher per game
5. Ranking pitchers by above metrics (e.g.: top 10, by team)


In [ ]:
#1. Avg. pitches thrown per game by pitcher
#Group by game_id, count pitches grouped pitcher
#Output: Total Pitches thrown by pitcher per game
total_pitches_pitcher = joined_atbats.groupby(by = ['g_id','pitcher_id'], as_index=False)['batter_id'].agg([np.count_nonzero]).reset_index()
total_pitches_pitcher = total_pitches_pitcher.sort_values(['g_id','count_nonzero','pitcher_id'], ascending=[True, False,True])

In [ ]:
total_pitches_pitcher.head(15)

In [ ]:
#example for one pitcher for all games
total_pitches_pitcher[total_pitches_pitcher['pitcher_id'] == 502239].sort_values(['g_id','count_nonzero'], ascending=[True, False]).head(5)

In [ ]:
#speed_viz3 = pd.merge(speed_viz2, event_count, how="left", on ='event')
# Average pitches thrown over all games (season)
# Output: Average pitches thrown per game
avg_pitches_pitcher = total_pitches_pitcher.groupby(['pitcher_id'], as_index=False)['count_nonzero'].agg([np.sum]).reset_index()
avg_pitches_pitcher = avg_pitches_pitcher.sort_values(['sum'], ascending=False)

In [ ]:
def metric_pitches_per_game_per_pitcher(joined_atbats):
    print('This metric returns Pitches thrown by a Pitcher per Game, per Season and Avg. Pitches thrown by a Pitcher per Game over a Season')
    print('\n')
    print('Enter Pitcher ID Number and Press Enter: ')
    pitcher_id = int(input())
    joined_atbats = joined_atbats
    
    #Search for that specific pitcher by ID
    joined_atbats = joined_atbats[joined_atbats['pitcher_id'] == pitcher_id]
    
    #Aggregate to get total pitches thrown by pitcher per game
    total_pitches_pitcher_game = joined_atbats.groupby(by = ['g_id','pitcher_id'], as_index=False)['batter_id'].agg([np.count_nonzero]).reset_index()
    total_pitches_pitcher_game = total_pitches_pitcher_game.sort_values(['g_id','count_nonzero','pitcher_id'], ascending=[True, False,True])
    total_pitches_pitcher_game = total_pitches_pitcher_game.sort_values(['g_id','count_nonzero'], ascending=[True, False])
    
    #Aggregate to get total pitches thrown by pitcher over season
    total_pitches_pitcher_season = total_pitches_pitcher_game.groupby(['pitcher_id'], as_index=False)['count_nonzero'].agg([np.sum]).reset_index()
    temp = total_pitches_pitcher_game.groupby(['pitcher_id'], as_index=False)['count_nonzero'].agg([np.sum]).reset_index()
    total_pitches_pitcher_season = temp.sort_values(['sum'], ascending=False)
    
    #Aggregate to get average pitches thrown by pitcher over season 
    total_games_pitcher = total_pitches_pitcher_game['g_id'].nunique()
    total_pitches_pitcher = total_pitches_pitcher_game['count_nonzero'].agg([np.sum])
    avg_pitches_pitcher = np.round(total_pitches_pitcher / total_games_pitcher, decimals = 0, out=None)
    
    #Return a DataFrame of Average Pitches 
    print('--------------------------------------------------------')
    print('For Pitcher ID ', pitcher_id, ':')
    print('Total Pitches Thrown per Game :')
    print(pd.DataFrame(total_pitches_pitcher_game))
    print('\n')
    print('Total Pitches Thrown per Season :', total_pitches_pitcher_season['sum'])
    print('\n')
    print('Average Pitches Thrown per Game :', avg_pitches_pitcher)
    print('--------------------------------------------------------')

In [ ]:
metric_pitches_per_game_per_pitcher(joined_atbats)

In [ ]:
joined_atbats[joined_atbats['pitcher_id'] == 502239]

In [ ]:
total_pitches_pitcher['g_id'].value_counts()

In [ ]:
### Analytics Project - Baseball Pitch by Pitch Analysis
#### Sikyun (George) Lee 
1. Obtain a Dataset
2. Model and store dataset (SQL, Pandas, ORM, Python Objects, etc.)
3. Visualize (Relationship, Insights, Descriptive Statstistics)
4. Think ahead of project
5. Due 11/25


#### Background information on data
Data is downloaded from Kaggle (https://www.kaggle.com/pschale/mlb-pitch-data-20152018?select=atbats.csv) but the source of this is from a webscrape in (http://gd2.mlb.com/components/game/mlb/). 

This data looks at pitch-level data from the 2015 to 2018 seasons in MLB. Through this data, I am hoping to answer some of these questions from a pitching standpoint.

**1. What are the pitch-types that gets outs the most? (unhittable pitches)**\
**2. Which pitcher has most pitches per inning ratio? (shows efficiency)**\
**3. Some other questions will asked as project progresses**\

For this session, I will some simple Exploratory Data Analysis to better understand and visualize the data

#### 1. Data source will be from the above ***mlb.com*** source through webscraping if possible. Currently, the link above is dead and if another substitute link cannot be found, the dataset from Kaggle will be used.

#For homework purposes, I'll only import and work with the 2019 season data

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#quick test
test = pd.read_csv('20tor.txt')

test.head(10)



#### 2. Store Data as Pandas Objects

atbats = pd.read_csv('2019_atbats.csv')
games = pd.read_csv('2019_games.csv')
pitches = pd.read_csv('2019_pitches.csv')
names = pd.read_csv('player_names.csv')

atbats.head()

games.head()

pitches.head()

#### 3. Visualize the Data for:
- Exploratory Data Analysis to find relationships and insights
- Descriptive Statistics

#### 3-1. Data Manipulation to Visualize the Data

atbats.columns

games.columns

pitches.columns

# What pitches were thrown the most frequently
# Join the pitches pitch_type column by the ab_id to the atbats data
joined_atbats = atbats.copy(deep = True)

joined_atbats = pd.merge(joined_atbats, pitches, how = 'left', on = 'ab_id')

joined_atbats.columns

a_viz = joined_atbats[['inning', 'top', 'ab_id', 'g_id', 'p_score', 'batter_id', 'pitcher_id',
             'p_throws', 'o', 'zone', 'pitch_type', 'b_count', 's_count', 'outs', 
              'pitch_num', 'on_1b', 'on_2b', 'on_3b']]

a_viz1 = a_viz['o_lag'] = a_viz['o'].shift(1)

a_viz['out'] = a_viz['o'] - a_viz['o_lag']

#Let's see who has on average and median thrown the most innings
a_viz1 = a_viz.groupby(by = 'pitcher_id', as_index=False).agg([np.mean])

a_viz1['pitcher_id'] = a_viz1.index

names['full_name'] = names.first_name + " " + names.last_name

#names = names.rename(columns = {'pitcher_id' : 'id'})

a_viz1['avg_inning'] = a_viz1['inning', 'mean']

a_viz1.sort_values('avg_inning', ascending=False)

a_viz1 = a_viz1.rename(columns = {'pitcher_id' : 'id'})

a_viz2 = a_viz1.head(10)

a_viz3 = pd.merge(a_viz2, names, how = 'left', on = 'id')

a_viz3['avg_inning'] = a_viz3['inning', 'mean']

a_viz3 = a_viz3.sort_values('avg_inning', ascending=False)

#### Output 1: Top 10 Pitchers with Highest Avg. Innings

# Barplot of top 10 pitchers with highest avg innings
sns.set(font_scale=2) 
f, ax = plt.subplots(figsize=(30, 15))
ax = sns.barplot(x="full_name", y="avg_inning", data=a_viz3)
ax.set(ylabel = "Avg. Innings", xlabel = "Pitcher", title = 'Top 10 Pitchers with Highest Avg. Innings')

b_viz1 = a_viz.groupby(by = ['pitch_type'], as_index=False).count()

b_viz1['counts'] = b_viz1['inning']

b_viz1

b_viz1 = b_viz1.sort_values('counts', ascending = False)

#### Output 2: Countplot of Pitch Types Thrown in 2019 Season

# Barplot of frequently thrown pitches in MLB 2019 season
sns.set(font_scale=3) 
f, ax = plt.subplots(figsize=(30, 15))
ax = sns.barplot(x="pitch_type", y="counts", data=b_viz1)
ax.set(ylabel = "Total Pitch Count", xlabel = "Pitch Type", title = 'Frequently Thrown Pitches')

a_viz.head(10)

pitch_out = a_viz.groupby(by = ['pitch_type'], as_index = False).sum()

pitch_out.head(10)

pitch_out_viz = pitch_out[['pitch_type', 'out']]

pitch_out_viz = pitch_out_viz.sort_values('out')

#### Output 3: Pitch Types that Converted an Out

# Barplot of frequently thrown pitches in MLB 2019 season
sns.set(font_scale=3) 
f, ax = plt.subplots(figsize=(30, 30))
ax = sns.barplot(x="pitch_type", y="out", data=pitch_out_viz)
ax.set(ylabel = "Total Outs Made", xlabel = "Pitch Type", title = 'Total Outs Converted by Pitch Types')

#### Next Steps

Although in Today's (11/24) session, Paul Carr has said that baseball players cannot use analytics for player valuation (i.e.: contract negotiation) due to arbitration reasons, the following analyses questions can possibly be answered with this data:\

**1. What kind of metrics can we define to measure a pitcher's "effciency?"**
This could be classic metrics such as pitches per inning or innings per inning, but novel metrics such as "number of runners on-base per game or inning" could something to think about.\
To take a step further, we can also look into "number of runs converted per runner on-base" to measure a pitcher's (including the team defense) ability to manage run-scoring situations.\

**2. How does a certain angle or speed of pitches convert into outs?**
For this, I haven't thought in detail in how I can measure this but considering that data for pitche speeds and angles are available, I am thinking of looking at how a range of pitch angles/speeds convert hitters into outs.

**3. Considering the strike/ball counts, how can we predict what pitcher will throw next?**
This would be predictive analytics that would probably be related to Bayesian Posterior probabilities, but will more time to think about this topic.

**4. Developing a function or package to read a pitcher's (probably starting pitcher) pitch data in an ongoing game and plot a heatmap of where that pitcher is pitching the most/least.**

#### 4. Analysis 

#### 4-1. Comparing Speed Changes by Pitch Type to Strikes
For each pitch type:
- How much does speed change? (This may be obvious for some types)
- How does the change of speed affect the number of strikes? (compared to balls, outs)

pitches.head()

joined_atbats.head()

speed = joined_atbats[['inning','ab_id','g_id','batter_id','pitcher_id','event','start_speed','end_speed','spin_rate','spin_dir',
               'break_angle','break_length','break_y','ax','ay','az','zone','pitch_type','b_score','b_count',
               's_count','outs','pitch_num','on_1b','on_2b','on_3b']]

speed['out'] = speed['outs'] - speed['outs'].shift(1)
#a_viz['out'] = a_viz['o'] - a_viz['o_lag']
#a_viz1 = a_viz['o_lag'] = a_viz['o'].shift(1)

speed['change_speed'] = speed['start_speed'] - speed['end_speed']

#change of speed per pitcher per game
speed_change_pitcher_game = speed.groupby(by = ['g_id','pitcher_id','pitch_type','event'])['change_speed'].mean()

#Plot a game's pitch type subbed with events to the average speed change 
speed_viz = speed.groupby(by = ['pitch_type','event'],as_index=False)['change_speed'].agg([np.mean])

speed_viz1 = speed_viz.reset_index()

#df.pivot_table(index='number', columns='class').swaplevel(axis=1).sort_index(1)#

speed_viz2 = speed.groupby(by = ['event'],as_index=False)['change_speed'].agg([np.mean]).reset_index()

event_count = speed.groupby(by = ['event'],as_index=False).count().reset_index()

event_count = event_count[['event','outs']]

event_count['counts'] = event_count['outs']

speed_viz2['event_cleaned'] = 0

speed_viz2.loc[0, 'event_cleaned'] = "Out"
speed_viz2.loc[1, 'event_cleaned'] = "Out"
speed_viz2.loc[2, 'event_cleaned'] = "Out"
speed_viz2.loc[3, 'event_cleaned'] = "Out"
speed_viz2.loc[4, 'event_cleaned'] = "Not Out"
speed_viz2.loc[5, 'event_cleaned'] = "Out"
speed_viz2.loc[6, 'event_cleaned'] = "Out"
speed_viz2.loc[7, 'event_cleaned'] = "Out"
speed_viz2.loc[8, 'event_cleaned'] = "Not Out"
speed_viz2.loc[9, 'event_cleaned'] = "Out"
speed_viz2.loc[10, 'event_cleaned'] = "Not Out"
speed_viz2.loc[11, 'event_cleaned'] = "Not Out"
speed_viz2.loc[12, 'event_cleaned'] = "Out"
speed_viz2.loc[13, 'event_cleaned'] = "Out"
speed_viz2.loc[14, 'event_cleaned'] = "Out"
speed_viz2.loc[15, 'event_cleaned'] = "Out"
speed_viz2.loc[16, 'event_cleaned'] = "Not Out"
speed_viz2.loc[17, 'event_cleaned'] = "Out"
speed_viz2.loc[18, 'event_cleaned'] = "Out"
speed_viz2.loc[19, 'event_cleaned'] = "Not Out"
speed_viz2.loc[20, 'event_cleaned'] = "Not Out"
speed_viz2.loc[21, 'event_cleaned'] = "Not Out"
speed_viz2.loc[22, 'event_cleaned'] = "Out"
speed_viz2.loc[23, 'event_cleaned'] = "Not Out"
speed_viz2.loc[24, 'event_cleaned'] = "Out"
speed_viz2.loc[25, 'event_cleaned'] = "Out"
speed_viz2.loc[26, 'event_cleaned'] = "Out"
speed_viz2.loc[27, 'event_cleaned'] = "Out"
speed_viz2.loc[28, 'event_cleaned'] = "Out"
speed_viz2.loc[29, 'event_cleaned'] = "Out"
speed_viz2.loc[30, 'event_cleaned'] = "Not Out"
speed_viz2.loc[31, 'event_cleaned'] = "Out"
speed_viz2.loc[32, 'event_cleaned'] = "Out"
speed_viz2.loc[33, 'event_cleaned'] = "Out"
speed_viz2.loc[34, 'event_cleaned'] = "Out"
speed_viz2.loc[35, 'event_cleaned'] = "Out"
speed_viz2.loc[36, 'event_cleaned'] = "Out"
speed_viz2.loc[37, 'event_cleaned'] = "Out"
speed_viz2.loc[38, 'event_cleaned'] = "Not Out"
speed_viz2.loc[39, 'event_cleaned'] = "Not Out"
speed_viz2.loc[40, 'event_cleaned'] = "Out"
speed_viz2.loc[41, 'event_cleaned'] = "Out"
speed_viz2.loc[42, 'event_cleaned'] = "Not Out"
speed_viz2.loc[43, 'event_cleaned'] = "Out"
speed_viz2.loc[44, 'event_cleaned'] = "Not Out"
speed_viz2.loc[45, 'event_cleaned'] = "Not Out"

speed_viz3 = pd.merge(speed_viz2, event_count, how="left", on ='event')

speed_viz3.drop(columns = ['outs'], inplace=True)

speed_viz3.sort_values('mean',ascending=False).head()

# Barplot of frequently thrown pitches in MLB 2019 season

sns.set(font_scale=3) 
f, ax = plt.subplots(figsize=(30, 15))
ax = sns.barplot(x="event", y="mean", data=speed_viz3, hue = 'event_cleaned')
ax.set(ylabel = "Total Count of Events", title = 'Number of Events')
ax.set_xticklabels(labels = speed_viz3['event'], rotation=45, fontsize=14)
ax.set(xlabel = "Events")

#Seems as though there isn't any big difference between speed change in Outs and Not Out results

To-Do:
1. Change of Speed per Inning and Change of Speed per Game for Pitchers (Look at the decrease in speed)
 - How does this correlate to an increase of Not Out situations (i.e.: hits, on base, etc.)
 How to do this:
 1) Pick a pitcher and get the average speed change per inning
  - Get speed change column and group by each inning to get average aggregation
 2) Calculate the number of average Not Out situations the pitcher makes per Inning and per Game
  - Make a Table of this per Inning and an overall Game
 3) Pick a pitcher and get the average speed change per game
  - Get speed change column and group by each inning to get average aggregation
2. Number of Pitches per Inning and Number of Pitches per Game for Pitchers (Look at the Trend)
 - How does the increase in pitches thrown correlate to the number of Not Out situations he has given 
 - Find a Threshold point of pitches thrown and decide when to pull out the pitcher
3. Pitches thrown in games over a season (could be for teams and/or specific players)
 - For teams, this could mean trouble as season progresses and may need some management 
 - For specific players, this may mean overusage and/or over-reliance on that specific player and may need management

pitches.head()

### Automatically show visualizaitons (plots) like ratretina since statcasts probably has all of them
### I think the pitches and hits by inning may be something good to look at for visualizations

### Part 2: Building Functions to Automatically Calculate Player Stats
1. Average Pitches thrown per game + visualization per pitcher per game
2. Average Strikes and Balls + Ratio by pitcher per game + visualization
3. Average Ground balls, Fly balls, Hits, On-base, Outs/On-Base Ratio by pitcher per game + visualization
4. Average Start_speed and End_speed, Start_speed:End_speed Ratio by Pitcher per game
5. Ranking pitchers by above metrics (e.g.: top 10, by team)


pitches.columns

joined_atbats.head()

#1. Avg. pitches thrown per game by pitcher
#Group by game_id, count pitches grouped pitcher
#Output: Total Pitches thrown by pitcher per game
total_pitches_pitcher = joined_atbats.groupby(by = ['g_id','pitcher_id'], as_index=False)['batter_id'].agg([np.count_nonzero]).reset_index()
total_pitches_pitcher = total_pitches_pitcher.sort_values(['g_id','count_nonzero','pitcher_id'], ascending=[True, False,True])

total_pitches_pitcher.head(15)

#example for one pitcher for all games
total_pitches_pitcher[total_pitches_pitcher['pitcher_id'] == 502239].sort_values(['g_id','count_nonzero'], ascending=[True, False]).head(5)

#speed_viz3 = pd.merge(speed_viz2, event_count, how="left", on ='event')
# Average pitches thrown over all games (season)
# Output: Average pitches thrown per game
avg_pitches_pitcher = total_pitches_pitcher.groupby(['pitcher_id'], as_index=False)['count_nonzero'].agg([np.sum]).reset_index()
avg_pitches_pitcher = avg_pitches_pitcher.sort_values(['sum'], ascending=False)

avg_pitches_pitcher.head(15)

def metric_pitches_per_game_per_pitcher(joined_atbats):
    print('This metric returns Pitches thrown by a Pitcher per Game, per Season and Avg. Pitches thrown by a Pitcher per Game over a Season')
    print('\n')
    print('Enter Pitcher ID Number and Press Enter: ')
    pitcher_id = int(input())
    joined_atbats = joined_atbats
    
    #Search for that specific pitcher by ID
    joined_atbats = joined_atbats[joined_atbats['pitcher_id'] == pitcher_id]
    
    #Aggregate to get total pitches thrown by pitcher per game
    total_pitches_pitcher_game = joined_atbats.groupby(by = ['g_id','pitcher_id'], as_index=False)['batter_id'].agg([np.count_nonzero]).reset_index()
    total_pitches_pitcher_game = total_pitches_pitcher_game.sort_values(['g_id','count_nonzero','pitcher_id'], ascending=[True, False,True])
    total_pitches_pitcher_game = total_pitches_pitcher_game.sort_values(['g_id','count_nonzero'], ascending=[True, False])
    
    #Aggregate to get total pitches thrown by pitcher over season
    total_pitches_pitcher_season = total_pitches_pitcher_game.groupby(['pitcher_id'], as_index=False)['count_nonzero'].agg([np.sum]).reset_index()
    temp = total_pitches_pitcher_game.groupby(['pitcher_id'], as_index=False)['count_nonzero'].agg([np.sum]).reset_index()
    total_pitches_pitcher_season = temp.sort_values(['sum'], ascending=False)
    
    #Aggregate to get average pitches thrown by pitcher over season 
    total_games_pitcher = total_pitches_pitcher_game['g_id'].nunique()
    total_pitches_pitcher = total_pitches_pitcher_game['count_nonzero'].agg([np.sum])
    avg_pitches_pitcher = np.round(total_pitches_pitcher / total_games_pitcher, decimals = 0, out=None)
    
    #Return a DataFrame of Average Pitches 
    print('--------------------------------------------------------')
    print('For Pitcher ID ', pitcher_id, ':')
    print('Total Pitches Thrown per Game :')
    print(pd.DataFrame(total_pitches_pitcher_game))
    print('\n')
    print('Total Pitches Thrown per Season :', total_pitches_pitcher_season['sum'])
    print('\n')
    print('Average Pitches Thrown per Game :', avg_pitches_pitcher)
    print('--------------------------------------------------------')

metric_pitches_per_game_per_pitcher(joined_atbats)

joined_atbats[joined_atbats['pitcher_id'] == 502239]

total_pitches_pitcher['g_id'].value_counts()

np.round()